 #  simple feed-forward neural network for MNIST

In [1]:
import tensorflow as tf
import math,os
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# dataset
data_sets = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


The fake_data flag is used for unit-testing purposes and may be safely ignored by the reader.

In [3]:
# placeholder
num_class = 10
images_placeholder = tf.placeholder(tf.float32, shape=(None,
                                                       28*28))
labels_placeholder = tf.placeholder(tf.int32, shape=(None,num_class))

In [4]:
# build graph
hidden_units = 100
weights = tf.Variable( tf.truncated_normal([28*28, hidden_units] , stddev = 1.0 / math.sqrt(float(28.28))))

biases = tf.Variable(tf.zeros([hidden_units]))

hidden1 = tf.nn.relu(tf.matmul(images_placeholder , weights) + biases)

weights_o = tf.Variable( tf.truncated_normal([hidden_units,num_class] , stddev = 1.0 / math.sqrt(float(28.28))))
biases_o = tf.Variable(tf.zeros([num_class]))

logits = tf.matmul(hidden1, weights_o) + biases_o

In [5]:
# loss
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=labels_placeholder, logits=logits))

In [6]:
# train and predict
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_placeholder, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
# 收集信息到tesorboard 和 保存
tf.summary.scalar('loss', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
summary = tf.summary.merge_all()

sess = tf.Session()
log_dir = './tmplog'
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
tf.gfile.MakeDirs(log_dir)
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)

saver = tf.train.Saver()

In [8]:
sess.run(tf.global_variables_initializer())
for i in range(1000):
    batch = data_sets.train.next_batch(50)
    feed_dict = {images_placeholder: batch[0], labels_placeholder: batch[1]}
    sess.run(train_step,feed_dict=feed_dict)    
    
    if (i + 1) % 100 == 0:
        summary_str,train_accuracy,loss = sess.run([summary,accuracy,cross_entropy], feed_dict=feed_dict)
        print('step %d, training accuracy %g , loss %g' % (i, train_accuracy,loss))

        summary_writer.add_summary(summary_str, i)
        summary_writer.flush()

        checkpoint_file = os.path.join(log_dir, 'model.ckpt')
        saver.save(sess, checkpoint_file, global_step=i)

batch_size = 50
batch_num = int(data_sets.test.num_examples / batch_size)
test_accuracy = 0

for i in range(batch_num):
    batch = data_sets.test.next_batch(batch_size)
    test_accuracy += sess.run(accuracy,feed_dict={images_placeholder: batch[0],
                                          labels_placeholder: batch[1]})

test_accuracy /= batch_num
print("test accuracy %g"%test_accuracy)

step 99, training accuracy 0.3 , loss 1.98607
step 199, training accuracy 0.44 , loss 1.68936
step 299, training accuracy 0.6 , loss 1.16975
step 399, training accuracy 0.68 , loss 1.07186
step 499, training accuracy 0.72 , loss 0.86204
step 599, training accuracy 0.84 , loss 0.69064
step 699, training accuracy 0.84 , loss 0.696578
step 799, training accuracy 0.72 , loss 0.766255
step 899, training accuracy 0.78 , loss 0.661721
step 999, training accuracy 0.92 , loss 0.387514
test accuracy 0.8569
